In [1]:
import tensorflow as tf
from models.EEGModels import EEGNet
import pandas as pd
import os
from datetime import timedelta
import numpy as np

from sklearn.model_selection import train_test_split

In [2]:
session_four_fp = os.path.join("data","session_four","_2021-03-10--03-38-49.csv")

ses4 = pd.read_csv(session_four_fp, parse_dates=["TimeStamp"])

# Time Markers
m1 = ses4.TimeStamp[0]
m2 = ses4.TimeStamp[0] + timedelta(seconds=60*5)
m3 = ses4.TimeStamp[0] + timedelta(seconds=60*10)
m4 = ses4.TimeStamp.iloc[-1]

# Data for EEGNet
ses4_timestamps = [m1, m2 , m3, m4]

In [3]:
# Coding note: this is a lot of hard-coding. 
# Data Preperation: Conversion to 128 Hz
# Concept
# group by second
# 1. drop na
# 2. sample random uniform 128
# 3. go to next group
initTime = ses4.TimeStamp[0]
one_second = timedelta(seconds=1)
finalTime = ses4.TimeStamp.iloc[-1]
df = pd.DataFrame([], columns=ses4.columns)
for t in range(1, (finalTime - initTime).seconds):
    start = initTime + one_second * (t - 1)
    end = start + one_second
    
    num_vals = ses4[(ses4.TimeStamp < end) & (ses4.TimeStamp > start)]
    num_vals.dropna(axis=1) # Drop na
    sample = num_vals.sample(128).sample(128, replace=False).sort_values("TimeStamp") # random sample
    df = df.append(sample)

feature_space = ['RAW_AF7', 'RAW_AF8', 'RAW_TP9', "RAW_TP10"]

# X should be formatted (trials, channels, samples, kernel=1)
X = np.array(df[feature_space])
temp = []
for i in range(928): # num of trials
    temp.append(X[i * 128:(i+1) * 128].T)
X = np.array(temp).reshape(928, 4, 128, 1)

y = np.concatenate([
    np.array([0,1] * 300).reshape(300, -2),
    np.array([1,0] * 300).reshape(300, -2),
    np.array([0,1] * 328).reshape(328, -2),
])

In [4]:
train, test_val = train_test_split(np.array(list(zip(X, y))), train_size=.9)
test, val = train_test_split(test_val, train_size=.5)

train_X, train_y = [np.array([ X for X, y in train ]), np.array([ y for X,y in train ])]
test_X, test_y = [np.array([ X for X, y in test ]), np.array([ y for X,y in test ])]
val_X, val_y = [np.array([ X for X, y in val ]), np.array([ y for X,y in val ])]

/home/nick/miniconda3/envs/cogs189_eegnet/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [5]:
model = EEGNet(
    nb_classes=2,
    Chans=4,
    Samples=128)
model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics=["accuracy"])
model.fit(train_X, train_y, validation_data=(val_X, val_y), batch_size=16, epochs=10)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 835 samples, validate on 47 samples
Epoch 1/10
835/835 [==============================] - 1s 1ms/sample - loss: nan - acc: 0.5000 - val_loss: nan - val_acc: 0.5000
Epoch 2/10
835/835 [==============================] - 1s 611us/sample - loss: nan - acc: 0.5000 - val_loss: nan - val_acc: 0.5000
Epoch 3/10
835/835 [==============================] - 1s 660us/sample - loss: nan - acc: 0.5000 - val_loss: nan - val_acc: 0.5000
Epoch 4/10
835/835 [==============================] - 1s 667us/sample - loss: nan - acc: 0.5000 - val_loss: nan - val_acc: 0.5000
Epoch 5/10
835/835 [==============================] - 1s 629us/sample - loss: nan - acc: 0.5000 - val_loss: nan - val_acc: 0.5000
Epoch 6/10
835/835 [==============================] - 1s 721us/sample - loss: nan - acc: 0.5000 - val_loss: nan - val_acc: 0.5000
Ep

In [6]:
model.predict(test_X)

array([[nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan]], dtype=float32)